In [1]:
from langgraph.graph import StateGraph, START, END
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_teddynote.graphs import visualize_graph

In [7]:
class State(TypedDict):
    input : Annotated[str,'query message']
    context : Annotated[list,add_messages]
    answer : Annotated[str,'result message']
    step : Annotated[str,'current step']
    is_evaluation : Annotated[bool,'current step']


In [8]:
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

In [9]:

def retriever(state : State):
    result = State({
        "context":"12313123",
        "input":state["input"],
        "step":"retriever"
        })
    return result

def is_evaluation(state : State):
    state['step']='is_evaluation'
    is_bool= False
    if is_bool:
        state['is_evaluation']=True
    else:
        state['is_evaluation']=False
    return state

In [10]:
state_graph = StateGraph(State)

state_graph.add_node('retriever',retriever)

state_graph.add_node('is_evaluation',is_evaluation)

state_graph.add_edge(START,'retriever')

state_graph.add_edge('retriever','is_evaluation')

state_graph.add_edge('is_evaluation',END)

graph = state_graph.compile()

mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retriever(retriever)
	is_evaluation(is_evaluation)
	__end__([<p>__end__</p>]):::last
	__start__ --> retriever;
	retriever --> is_evaluation;
	is_evaluation --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [11]:
###### STEP 7. 그래프 실행 ######
question = "서울의 유명한 맛집 TOP 10 추천해줘"

for event in graph.stream({"input": [("user", question)]}):
    # 이벤트 값 출력
    for value in event.values():
        print("="*30)
        print(value)
        # print(value["input"][-1].content)

{'context': '12313123', 'input': [('user', '서울의 유명한 맛집 TOP 10 추천해줘')], 'step': 'retriever'}
{'input': [('user', '서울의 유명한 맛집 TOP 10 추천해줘')], 'context': [HumanMessage(content='12313123', additional_kwargs={}, response_metadata={}, id='2307691f-0e79-419c-bf50-e6f07a832b20')], 'step': 'is_evaluation', 'is_evaluation': False}
